In [ ]:
!pip install huggingface_hub transformers

In [ ]:
from huggingface_hub import notebook_login
from transformers import pipeline

notebook_login()

In [ ]:
classifier = pipeline("ner", model="token-classifier/roBERTa-v2", aggregation_strategy="simple")

In [ ]:
classifier("A user is authenticated by the server using a login and password.")

In [ ]:
import json, concurrent.futures

main = []
output = []

with open("processed\\main.json", encoding="utf-8") as jp:
    tmp = json.load(jp)

    for i in tmp:
        main.append(" ".join(i["tokens"]))

def batch_fn(args):
    num, arr = args
    idx = 0
    print(f"Batch#{num} running...")
    for i in arr:
        print(f"Batch#{num} @ {idx}...")

        op = classifier(i)
        output.append({"spec": i, "extraction": op})

        idx += 1

    print(f"Batch {num} done")
    return

output = []
divs = 10
batches = [(i, i*(len(main)//divs), (i+1)*(len(main)//divs)) for i in range(divs)]

executor = concurrent.futures.ThreadPoolExecutor(max_workers=len(batches))
list(executor.map(batch_fn, batches))

In [ ]:
output

In [ ]:
for i in output:
    for j in i["extraction"]:
        j["score"] = float(j["score"])

with open("results.json", "w", encoding="utf-8") as jp:
    json.dump(output, jp, indent=4)

In [ ]:
dist = {
    "no": 0,
    "class": 0,
    "method": 0,
    "attribute": 0,
    "association": 0,
    "generalization": 0,
}

for i in output:
    for j in i["extraction"]:
        dist[j["entity_group"]] += 1

print(dist)

with open("extractions.json", "w") as jp:
    json.dump(dist, jp, indent=4)